# Extrair comentários do doctoralia 

* **Objetivo:** Extrair o comentário de todos os médicos psiquiatras de SCJ

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install chromedriver-autoinstaller
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()

#pip install webdriver-managerr

In [96]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from unidecode import unidecode
import time
import pandas as pd

## Nome dos medicos das 5 primeiras paginas

In [113]:
urls = [
    'https://www.doctoralia.com.br/pesquisa?q=Psiquiatra&loc=São%20José%20dos%20Campos%2C%20SP&filters%5Bspecializations%5D%5B%5D=78',
   ]


In [114]:
# Configuração do Selenium WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

def names_medicos(list_urls):
    list_doctors = []
    
    for url in list_urls:
        driver.get(url)
        
        # Espera até que o elemento desejado esteja presente no DOM
        try:
            element_present = EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span[itemprop='name']"))
            WebDriverWait(driver, 20).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
            continue
        
        # Obtém o conteúdo da página
        html_content = driver.page_source
        
        # Analisa o conteúdo HTML com BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Encontra todos os elementos <span> com o atributo itemprop='name'
        names = soup.find_all('span', itemprop='name')
        
        # Extrai os textos dos elementos encontrados e adiciona à lista
        for name in names:
            list_doctors.append(name.get_text(strip=True))
    
    # Fecha o navegador
    driver.quit()
    
    return list_doctors


doctor_names = names_medicos(urls)



In [117]:
# Aplicar unidecode para remover a acentuação e converter para minúsculas
medicos = [unidecode(i).lower().removeprefix("dra. ").removeprefix("dr. ") for i in doctor_names]

# Definir especialidade e cidade
BASE_DOMAIN = "https://www.doctoralia.com.br"
SPECIALIZATION = "pisciquiatra"
CITY = "sao-jose-dos-campos"

# Gerar lista de URLs
urls = ["{}/{}/{}/{}".format(BASE_DOMAIN, '-'.join(nome.lower().split()), SPECIALIZATION, CITY) for nome in medicos]

# Criar DataFrame
list_pg = pd.DataFrame({
    'DOCTORS': medicos,
    'urls': urls
})

print(list_pg.shape)
list_pg.head(10)


(20, 2)


,DOCTORS,urls
0,maira felix,https://www.doctoralia.com.br/maira-felix/pisc...
1,luiz guilherme de siqueira flora,https://www.doctoralia.com.br/luiz-guilherme-d...
2,gusttavo fernandes,https://www.doctoralia.com.br/gusttavo-fernand...
3,lais kristyna rocha de oliveira,https://www.doctoralia.com.br/lais-kristyna-ro...
4,tiago miranda,https://www.doctoralia.com.br/tiago-miranda/pi...
5,natalia belsuzarri,https://www.doctoralia.com.br/natalia-belsuzar...
6,maressa carvalho,https://www.doctoralia.com.br/maressa-carvalho...
7,daiana melo,https://www.doctoralia.com.br/daiana-melo/pisc...
8,francisco jose hennemann neto,https://www.doctoralia.com.br/francisco-jose-h...
9,ariel cesar de carvalho,https://www.doctoralia.com.br/ariel-cesar-de-c...


In [120]:
# Inicializa o driver do Chrome (certifique-se de ter o ChromeDriver instalado e em seu PATH)
driver = webdriver.Chrome()

# Função para clicar em "Ver mais" um determinado número de vezes
def clicar_ver_mais(vezes):
    for _ in range(vezes):
        try:
            ver_mais_button = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//a[contains(text(),'Veja mais')]")))
            driver.execute_script("arguments[0].scrollIntoView();", ver_mais_button)
            ver_mais_button.click()
            time.sleep(5)  # Aguarda um curto intervalo para carregar mais comentários
        except:
            break

# Função para coletar os comentários de uma URL específica
def coletar_comentarios(url):
    driver.get(url)
    clicar_ver_mais(100) # Clicar em "Ver mais" 10 vezes

    comentarios = driver.find_elements(By.CLASS_NAME, 'text-break')
    lista_comentarios = []

    for comentario in comentarios:
        lista_comentarios.append(comentario.text.strip())

    return lista_comentarios

# DataFrame df1 com as colunas "medico" e "url"
# Vamos supor que este DataFrame já esteja preenchido com os dados necessários
df1 = list_pg

# Listas para armazenar os resultados
lista_medicos = []
lista_comentarios_total = []

for index, row in list_pg.iterrows():
    medico = row['DOCTORS']
    url = row['urls']

    comentarios = coletar_comentarios(url)

    lista_medicos.extend([medico] * len(comentarios))
    lista_comentarios_total.extend(comentarios)

# Fecha o navegador após coletar todos os comentários
driver.quit()

# Criar DataFrame com os resultados
df = pd.DataFrame({
    'medico': lista_medicos,
    'comentarios': lista_comentarios_total
})

In [123]:
df['medico'].value_counts()

medico
maira felix                         20
luiz guilherme de siqueira flora    20
gusttavo fernandes                  20
lais kristyna rocha de oliveira     20
natalia belsuzarri                  20
maressa carvalho                    20
francisco jose hennemann neto       20
ariel cesar de carvalho             20
rogers diego ascef                  20
eduardo rosadas barbosa machado     20
gisele amarante pereira             20
ana paula werberich lange           20
natalia gorgete                     20
evellyn alcanfor                    20
renata goncalves                     6
Name: count, dtype: int64

In [124]:
df.head(5)

,medico,comentarios
0,maira felix,S\nSilvia Maciel\nVERIFICADO PELA DOCTORALIA\n...
1,maira felix,Excelente profissional...Tratamento muito efic...
2,maira felix,B\nBeatriz\nVERIFICADO PELA DOCTORALIA\n24 de ...
3,maira felix,"Otima e atenciosa, ficou bastante tempo com es..."
4,maira felix,W\nWellington mamedes\nVERIFICADO PELA DOCTORA...


In [125]:
from dateutil import parser

# Mapeamento dos nomes dos meses em português para inglês
meses_pt_para_en = {
    'janeiro': 'January',
    'fevereiro': 'February',
    'março': 'March',
    'abril': 'April',
    'maio': 'May',
    'junho': 'June',
    'julho': 'July',
    'agosto': 'August',
    'setembro': 'September',
    'outubro': 'October',
    'novembro': 'November',
    'dezembro': 'December'
}

# Função para extrair datas da string
def extrair_data(texto):
    for mes_pt, mes_en in meses_pt_para_en.items():
        texto = texto.replace(mes_pt, mes_en)
    try:
        return parser.parse(texto, fuzzy=True)
    except ValueError:
        return pd.NA

# Aplicar a função à coluna 'Texto' para extrair as datas
df['Data'] = df['comentarios'].apply(extrair_data)
df['del'] = df['Data'].isna().astype(int)
df['Data'] = df['Data'].fillna(method='ffill')
df = df[df['del'] != 0]
df.head(3)

/var/folders/jz/dk0jb_5x4ms816sj1m0xpqxw0000gn/T/ipykernel_4657/2049312619.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Data'] = df['Data'].fillna(method='ffill')
/var/folders/jz/dk0jb_5x4ms816sj1m0xpqxw0000gn/T/ipykernel_4657/2049312619.py:31: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Data'] = df['Data'].fillna(method='ffill')


In [38]:
#df.to_csv("data.frame/comentarios_por_medico.csv", index=False)
df = pd.read_csv("data.frame/comentarios_por_medico.csv")
df['Data'] = df['Data'].fillna(method='ffill')


/var/folders/jz/dk0jb_5x4ms816sj1m0xpqxw0000gn/T/ipykernel_4657/3491403682.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Data'] = df['Data'].fillna(method='ffill')
